Dijkstra's algorithm

In [ ]:
!pip install requests
!pip install pandas

In [35]:
import requests
import pandas as pd

# Matrix[n_row][n_column]["name"] or ["coordinates"]
def build_matrix(rows, columns):
    url = "https://photon.komoot.io/api/"
    matrix = []

    for row in rows:
        row_number = int(row.split()[0][0])
        row_list = []

        for col in columns:
            if isinstance(col, tuple):
                col_name = col[1] if 5 <= row_number <= 7 else col[0]
            else:
                col_name = col

            query = f"{row} {col_name} Antigua Guatemala"
            params = {"q": query, "limit": 1}
            response = requests.get(url, params=params)
            data = response.json()

            if data.get("features"):
                lat = data["features"][0]["geometry"]["coordinates"][1]
                lon = data["features"][0]["geometry"]["coordinates"][0]
            else:
                lat = None
                lon = None

            row_list.append({"name" : (row, col_name), "coordinates" : (lat, lon)})

        matrix.append(row_list)

    return matrix

rows = [
    "1a Calle Poniente",
    "2a Calle Poniente",
    "3a Calle Poniente",
    "4a Calle Poniente",
    "5a Calle Poniente",
    "6a Calle Poniente",
    "7a Calle Poniente"
]

columns = [
    ("Alameda Santa Lucia", "Calzada Santa Lucia Sur"),
    "7a Avenida Norte",
    "6a Avenida Norte",
    "5a Avenida Norte",
    "4a Avenida Norte",
    "3a Avenida Norte",
    "2a Avenida Norte",
    "1a Avenida Norte"
]

matrix_map_info = build_matrix(rows, columns)

In [49]:
print(f"{matrix_map_info[0][0]}")

{'name': ('1a Calle Poniente', 'Alameda Santa Lucia'), 'coordinates': (14.560572, -90.7392661)}
